In [ ]:
#links referred:
# https://www.kaggle.com/jonaspptawat/cyberbullying-classification-eda-and-ml

In [1]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import string
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
tweets = pd.read_csv("cyberbullying_tweets.csv")

In [60]:
tweets

,Unnamed: 0,tweet_text,cyberbullying_type,processed_tweet
0,0,"In other words #katandandre, your food was cra...",not_cyberbullying,words katandandre food crapilicious mkr
1,1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,aussietv white mkr theblock imacelebrityau tod...
2,2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,classy whore red velvet cupcakes
3,3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,meh p thanks heads not concerned another ang...
4,4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,isis account pretending kurdish account like...
...,...,...,...,...
47687,47687,"Black ppl aren't expected to do anything, depe...",ethnicity,black ppl expected anything depended anything ...
47688,47688,Turner did not withhold his disappointment. Tu...,ethnicity,turner not withhold disappointment turner call...
47689,47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,swear god dumb nigger bitch got bleach hair re...
47690,47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,yea fuck rt therealexel youre nigger fucking u...


In [5]:
tweets['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [6]:
#remove stopwords
stop_words = set(stopwords.words('english'))
stop_words.remove("not")
stop_words.add("rt") #for twitter retweet
stop_words.remove("against")
# stop_words

In [7]:
twitter_string = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [26]:
tweets["processed_tweet"] = tweets["tweet_text"].str.lower()

In [28]:
#remove tags to people and 
# retweet links of the people 
# and also remove stopwords
# and convert emoji to words

tweets["processed_tweet"] = tweets["processed_tweet"] \
                            .apply(lambda x: re.sub(twitter_string,' ', x)) \
                            .apply(lambda x : [value for value in x.split(" ") if not value.startswith("@") and value not in stop_words]) \
                            .apply(lambda x : ' '.join(x)) \
                            .apply(lambda x : x.translate(str.maketrans('', '', string.punctuation))) \
                            .apply(lambda x: emoji.demojize(x)) 
                        

In [10]:
tweets.to_csv("Intermediate_file.csv")

In [40]:
tweets = pd.read_csv("Intermediate_file.csv")

In [ ]:
#extracting sentiment of each tweet

In [41]:
sid_obj = SentimentIntensityAnalyzer()

In [42]:
tweets['sentiments'] = tweets['processed_tweet'].apply(lambda x: sid_obj.polarity_scores(str(x)))

In [43]:
tweet_sentiment = tweets['sentiments'].apply(pd.Series)

In [45]:
tweets = tweets.merge(tweet_sentiment, left_index=True, right_index=True)

In [48]:
tweets.rename(columns = {"neu":"Neutral", "negative":"Negative", "pos":"Positive", "compound":"Total_Sentiment_Score"}, inplace=True)

In [49]:
tweets

,Unnamed: 0,tweet_text,cyberbullying_type,processed_tweet,sentiments,neg,Neutral,Positive,Total_Sentiment_Score
0,0,"In other words #katandandre, your food was cra...",not_cyberbullying,words katandandre food crapilicious mkr,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
1,1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,aussietv white mkr theblock imacelebrityau tod...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
2,2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,classy whore red velvet cupcakes,"{'neg': 0.422, 'neu': 0.294, 'pos': 0.284, 'co...",0.422,0.294,0.284,-0.3400
3,3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,meh p thanks heads not concerned another ang...,"{'neg': 0.094, 'neu': 0.504, 'pos': 0.403, 'co...",0.094,0.504,0.403,0.6488
4,4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,isis account pretending kurdish account like...,"{'neg': 0.239, 'neu': 0.427, 'pos': 0.333, 'co...",0.239,0.427,0.333,0.0258
...,...,...,...,...,...,...,...,...,...
47687,47687,"Black ppl aren't expected to do anything, depe...",ethnicity,black ppl expected anything depended anything ...,"{'neg': 0.165, 'neu': 0.45, 'pos': 0.384, 'com...",0.165,0.450,0.384,0.8074
47688,47688,Turner did not withhold his disappointment. Tu...,ethnicity,turner not withhold disappointment turner call...,"{'neg': 0.097, 'neu': 0.823, 'pos': 0.079, 'co...",0.097,0.823,0.079,-0.1526
47689,47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,swear god dumb nigger bitch got bleach hair re...,"{'neg': 0.643, 'neu': 0.275, 'pos': 0.082, 'co...",0.643,0.275,0.082,-0.9356
47690,47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,yea fuck therealexel youre nigger fucking unfo...,"{'neg': 0.625, 'neu': 0.375, 'pos': 0.0, 'comp...",0.625,0.375,0.000,-0.9128


In [ ]:
#topic and aspect extraction from each tweet

In [ ]:
#TFIdf vectorizer